<h1><center><font size="6">Data Cleaning, EDA, and Feature Engineering Notebook</font></center></h1>

# Contents

- <a href='#1'>Importing Packages</a>  
- <a href='#2'>Uploading Data</a>
- <a href='#3'>Creating Train, Validation, and Testing Sets</a>  
- <a href='#4'>Data Cleaning</a>  
- <a href='#5'>Exploratory Data Analysis</a>
- <a href='#6'>Feature Engineering</a>
- <a href='#7'>Post-Feature Selection</a> 

# <a id='1'>Importing Packages</a>

In [2]:
# Importing Packages
import numpy as np 
import pandas as pd
import re
import json
import requests
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)

import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
%matplotlib inline
plt.style.use("fivethirtyeight")

from sklearn.model_selection import train_test_split
import pickle

# <a id='2'>Uploading Data</a>

In [3]:
df = pd.read_excel("../data/default of credit card clients.xls")
new_header = df.iloc[0]
df = df[1:] 
df.columns = new_header
df = df.rename(columns={"default payment next month": "default"}) 
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
1,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [4]:
df.dtypes

0
ID           object
LIMIT_BAL    object
SEX          object
EDUCATION    object
MARRIAGE     object
AGE          object
PAY_0        object
PAY_2        object
PAY_3        object
PAY_4        object
PAY_5        object
PAY_6        object
BILL_AMT1    object
BILL_AMT2    object
BILL_AMT3    object
BILL_AMT4    object
BILL_AMT5    object
BILL_AMT6    object
PAY_AMT1     object
PAY_AMT2     object
PAY_AMT3     object
PAY_AMT4     object
PAY_AMT5     object
PAY_AMT6     object
default      object
dtype: object

# <a id='4'>Creating Train, Validation, and Testing Sets</a>

In [5]:
X = df.drop(["default"], axis=1)
y = df["default"]
# split the full data 80:20 into training:validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=42)
# split training data 87.5:12.5 into training:testing sets
X_tr, X_tt, y_tr, y_tt = train_test_split(X_train, y_train, train_size=0.875, random_state=42)
train = pd.concat([X_tr, y_tr], axis=1)
val = pd.concat([X_val, y_val], axis=1)

In [6]:
# X_tt.to_csv("../data/testing.csv")
# y_tt.to_csv("../data/testing_labels.csv")
# train.to_csv("../data/training.csv", index=False)
# val.to_csv("../data/validate.csv", index=False)

# <a id='5'>Data Cleaning</a>

In [35]:
tr = pd.read_csv("../data/training.csv")
val = pd.read_csv("../data/validate.csv")
tr = tr.drop(["ID"], axis=1)
val = val.drop(["ID"], axis=1)
tr.dtypes

LIMIT_BAL    int64
SEX          int64
EDUCATION    int64
MARRIAGE     int64
AGE          int64
PAY_0        int64
PAY_2        int64
PAY_3        int64
PAY_4        int64
PAY_5        int64
PAY_6        int64
BILL_AMT1    int64
BILL_AMT2    int64
BILL_AMT3    int64
BILL_AMT4    int64
BILL_AMT5    int64
BILL_AMT6    int64
PAY_AMT1     int64
PAY_AMT2     int64
PAY_AMT3     int64
PAY_AMT4     int64
PAY_AMT5     int64
PAY_AMT6     int64
default      int64
dtype: object

In [36]:
tr.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,50000,2,2,1,44,0,0,0,0,0,0,45578,41906,35703,22360,23659,27421,3016,5003,3000,3000,5000,924,0
1,160000,2,3,1,46,-1,-1,-1,0,-1,-1,24904,2338,4856,4127,3995,842,2343,4856,1000,3995,842,26313,0
2,100000,2,2,1,47,-1,-1,-1,-1,-1,-2,6666,6666,0,6270,-396,-396,6666,0,6270,0,0,0,1
3,170000,2,2,1,29,0,0,0,0,0,0,79091,62575,63317,63903,43505,43995,2235,2493,2585,1683,1901,2111,0
4,150000,2,1,2,33,-2,-2,-2,-2,-2,-2,24393,26847,32702,33459,27800,2261,27007,32715,33475,27808,2261,169465,0


In [37]:
# look for anomalies in minimum and maximum values and peculiarities in interquartile values
tr.describe()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
count,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,2.100000e+04,21000.000000,21000.000000,21000.000000,21000.000000,2.100000e+04,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000
mean,167214.937143,1.605190,1.854333,1.549476,35.538286,-0.015524,-0.130952,-0.165667,-0.216952,-0.261238,-0.284810,51115.879143,49164.750619,4.696772e+04,43174.979429,40376.466143,38976.187810,5732.100667,6.006080e+03,5279.434524,4927.353714,4822.283381,5156.729952,0.221714
std,129561.159854,0.488821,0.794086,0.523126,9.257936,1.127750,1.200066,1.194393,1.170611,1.136246,1.156573,73469.974615,71175.850265,6.960624e+04,64284.893891,61004.334726,59676.470266,17510.937498,2.508609e+04,18647.406220,16803.515819,15620.014696,17658.819827,0.415410
min,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-165580.000000,-69777.000000,-1.572640e+05,-170000.000000,-81334.000000,-209051.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3566.500000,3033.500000,2.743750e+03,2350.750000,1782.000000,1301.750000,1000.000000,8.360000e+02,390.000000,291.000000,243.000000,116.750000,0.000000
50%,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22460.500000,21417.000000,2.007300e+04,19038.500000,18100.000000,17128.500000,2100.000000,2.007000e+03,1800.000000,1500.000000,1500.000000,1500.000000,0.000000
75%,240000.000000,2.000000,2.000000,2.000000,42.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,66759.750000,63291.250000,5.969425e+04,54345.500000,50289.250000,49413.000000,5007.000000,5.000000e+03,4500.000000,4001.000000,4034.000000,4000.000000,0.000000
max,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,964511.000000,983931.000000,1.664089e+06,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.000000,621000.000000,417990.000000,527143.000000,1.000000


In [38]:
# Use requests and json modules to webscrape current exchange rate for TWD to USD
url = 'https://openexchangerates.org/api/latest.json?app_id=c51b1508fb4145259b1c2fade72a2c04'
response = requests.get(url)
data = response.json()
rate = data['rates']['TWD']

In [39]:
# check for null values
data = [tr, val]
for d in data:
    print(d.isna().sum())

LIMIT_BAL    0
SEX          0
EDUCATION    0
MARRIAGE     0
AGE          0
PAY_0        0
PAY_2        0
PAY_3        0
PAY_4        0
PAY_5        0
PAY_6        0
BILL_AMT1    0
BILL_AMT2    0
BILL_AMT3    0
BILL_AMT4    0
BILL_AMT5    0
BILL_AMT6    0
PAY_AMT1     0
PAY_AMT2     0
PAY_AMT3     0
PAY_AMT4     0
PAY_AMT5     0
PAY_AMT6     0
default      0
dtype: int64
LIMIT_BAL    0
SEX          0
EDUCATION    0
MARRIAGE     0
AGE          0
PAY_0        0
PAY_2        0
PAY_3        0
PAY_4        0
PAY_5        0
PAY_6        0
BILL_AMT1    0
BILL_AMT2    0
BILL_AMT3    0
BILL_AMT4    0
BILL_AMT5    0
BILL_AMT6    0
PAY_AMT1     0
PAY_AMT2     0
PAY_AMT3     0
PAY_AMT4     0
PAY_AMT5     0
PAY_AMT6     0
default      0
dtype: int64


In [40]:
# change column names for easier reference
for d in data:    
    d.rename(columns={"PAY_0": "behind1", 
                        "PAY_2": "behind2", 
                        "PAY_3": "behind3", 
                        "PAY_4": "behind4",
                        "PAY_5": "behind5", 
                        "PAY_6": "behind6", 
                        "BILL_AMT1": "billed1", 
                        "BILL_AMT2": "billed2", 
                        "BILL_AMT3": "billed3", 
                        "BILL_AMT4": "billed4", 
                        "BILL_AMT5": "billed5", 
                        "BILL_AMT6": "billed6", 
                        "PAY_AMT1": "paid1", 
                        "PAY_AMT2": "paid2", 
                        "PAY_AMT3": "paid3", 
                        "PAY_AMT4": "paid4", 
                        "PAY_AMT5": "paid5", 
                        "PAY_AMT6": "paid6",
                        "SEX": "gender",
                        "EDUCATION": "education",
                        "MARRIAGE": "marriage",
                        "AGE": "age",
                        "LIMIT_BAL": "limit"}, inplace=True)

In [41]:
# Change all Taiwanese to US Dollars for better understanding for American audience
for d in data:
    d[['limit']] = d[['limit']]/rate
    d[['billed1', 'billed2', 'billed3', 'billed4', 'billed5', 'billed6']] = d[['billed1', 'billed2', 'billed3', 'billed4', 'billed5', 'billed6']].divide(rate, axis=1)
    d[['paid1', 'paid2', 'paid3', 'paid4', 'paid5', 'paid6']] = d[['paid1', 'paid2', 'paid3', 'paid4', 'paid5', 'paid6']].divide(rate, axis=1)

In [42]:
# Round all decimals to two decimal places to represent cents
for d in data:
    d['limit'] = d['limit'].apply(lambda x: round(x, 2))
    d[['billed1', 'billed2', 'billed3', 'billed4', 'billed5', 'billed6']] = d[['billed1', 'billed2', 'billed3', 'billed4', 'billed5', 'billed6']].apply(lambda x: round(x, 2))
    d[['paid1', 'paid2', 'paid3', 'paid4', 'paid5', 'paid6']] = d[['paid1', 'paid2', 'paid3', 'paid4', 'paid5', 'paid6']].apply(lambda x: round(x, 2))

In [43]:
# put all zero values into category of 3 (other) for marriage
# lump all the other and unknown education categories together:  0, 5, 6 values to category 4 (other)
for d in data:  
    d.replace({'marriage': {0:3}}, inplace=True)
    d.replace({'education': {5:4, 0:4, 6:4}}, inplace=True)

## Pickle out baseline models

In [44]:
tr.head()

,limit,gender,education,marriage,age,behind1,behind2,behind3,behind4,behind5,behind6,billed1,billed2,billed3,billed4,billed5,billed6,paid1,paid2,paid3,paid4,paid5,paid6,default
0,1790.26,2,2,1,44,0,0,0,0,0,0,1631.93,1500.45,1278.35,800.60,847.12,981.81,107.99,179.13,107.42,107.42,179.03,33.08,0
1,5728.83,2,3,1,46,-1,-1,-1,0,-1,-1,891.69,83.71,173.87,147.77,143.04,30.15,83.89,173.87,35.81,143.04,30.15,942.14,0
2,3580.52,2,2,1,47,-1,-1,-1,-1,-1,-2,238.68,238.68,0.00,224.50,-14.18,-14.18,238.68,0.00,224.50,0.00,0.00,0.00,1
3,6086.88,2,2,1,29,0,0,0,0,0,0,2831.87,2240.51,2267.08,2288.06,1557.71,1575.25,80.02,89.26,92.56,60.26,68.07,75.58,0
4,5370.78,2,1,2,33,-2,-2,-2,-2,-2,-2,873.40,961.26,1170.90,1198.01,995.38,80.96,966.99,1171.37,1198.58,995.67,80.96,6067.73,0


In [45]:
pickle_out = open("../data/training_cleaned.pickle","wb")
pickle.dump(tr, pickle_out)
pickle_out.close()

In [46]:
pickle_out = open("../data/validate_cleaned.pickle","wb")
pickle.dump(val, pickle_out)
pickle_out.close()

# <a id='6'>Exploratory Data Analysis</a>

In [47]:
# organize features into categorical and continuous
categorical = tr[['gender', 'marriage', 'education', 'behind1', 'behind2', 'behind3', 'behind4', 'behind5', 'behind6']]
continuous = tr[['limit', 'age', 'billed1', 'billed2', 'billed3', 'billed4', 'billed5', 'billed6', 'paid1', 'paid2', 'paid3', 'paid4', 'paid5', 'paid6']]
cat_col = categorical.columns
cont_col = continuous.columns

In [48]:
# display distributions of all the continuous variables

# con_1 = pd.melt(tr, value_vars = cont_col)
# sns.set_theme(style="darkgrid", font='serif', context='talk')
# g = sns.FacetGrid(con_1, col='variable', col_wrap=3, sharex=False, sharey=False, height=4)
# g = g.map(sns.distplot, 'value', color='r')
# g.set_xticklabels(rotation=45)
# g.fig.subplots_adjust(top=0.9)
# g.fig.suptitle("Distributions of Continuous Features")
# g.fig.tight_layout()
# plt.savefig("../images/distplot.png")

<img src="../images/distplot.png">

In [49]:
# Use bar graphs of the distribution of data for categorical variables

# cat_1 = pd.melt(tr, value_vars=cat_col)
# sns.set_theme(style="darkgrid", font='serif', context='talk')
# g = sns.FacetGrid(cat_1, col='variable', col_wrap=3, sharex=False, sharey=False, height=4)
# g = g.map(sns.countplot, 'value', color='dodgerblue')
# g.set_xticklabels()
# g.fig.subplots_adjust(top=0.9)
# g.fig.suptitle("Distributions of Categorical Features")
# g.fig.tight_layout()
# plt.savefig("../images/countplot.png")

<img src="../images/countplot.png">

In [50]:
yes = tr.default.sum()
no = len(tr)-yes
perc_y = round(yes/len(tr)*100, 1)
perc_n = round(no/len(tr)*100, 1)

# plt.figure(figsize=(8,6))
# sns.set_theme(style="darkgrid", font='serif', context='talk')
# sns.countplot('default', data=tr)
# plt.title('Credit Card Baseline Default', size=16)
# plt.box(False);
# plt.savefig("../images/baseline.png")

<img src="../images/baseline.png">

In [51]:
print("Number of Total Non-Defaulters: ", yes)
print("Number of Defaulters: ", no)
print("Percentage of Non-Defaulters: ", perc_y)
print("Percentage of Defaulters: ", perc_n)

pd.DataFrame
default = pd.DataFrame(data = {"Training Dataset": [yes, no, perc_y, perc_n]}, 
                       index = ["Number of Total Non-Defaulters: ", "Number of Defaulters: ", "Percentage of Non-Defaulters: ", "Percentage of Defaulters: "])
default

Number of Total Non-Defaulters:  4656
Number of Defaulters:  16344
Percentage of Non-Defaulters:  22.2
Percentage of Defaulters:  77.8


,Training Dataset
Number of Total Non-Defaulters:,4656.0
Number of Defaulters:,16344.0
Percentage of Non-Defaulters:,22.2
Percentage of Defaulters:,77.8


In [52]:
# subset = tr[['gender', 'education', 'marriage', 'behind1', 'behind2', 'behind3', 'behind4', 'behind5', 'behind6', 'default']]
# f, axes = plt.subplots(3, 3, figsize=(15, 12), facecolor='white')
# sns.set_theme(style="darkgrid", font='serif', context='paper')
# f.suptitle('Frequency of Categorical Variables', size=16)
# ax1 = sns.countplot(x="gender", hue="default", data=subset, ax=axes[0,0])
# ax2 = sns.countplot(x="education", hue="default", data=subset, ax=axes[0,1])
# ax3 = sns.countplot(x="marriage", hue="default", data=subset, ax=axes[0,2])
# ax4 = sns.countplot(x="behind1", hue="default", data=subset, ax=axes[1,0])
# ax5 = sns.countplot(x="behind2", hue="default", data=subset, ax=axes[1,1])
# ax6 = sns.countplot(x="behind3", hue="default", data=subset, ax=axes[1,2])
# ax7 = sns.countplot(x="behind4", hue="default", data=subset, ax=axes[2,0])
# ax8 = sns.countplot(x="behind5", hue="default", data=subset, ax=axes[2,1])
# ax9 = sns.countplot(x="behind6", hue="default", data=subset, ax=axes[2,2])
# plt.savefig("../images/default_freq_by_cat.png")

<img src="../images/default_freq_by_cat.png">

In [53]:
# plt.figure(figsize = (14,6))
# plt.title('Distribution of Credit Limit', size=16)
# sns.set_theme(style="darkgrid", font='serif', context='poster')
# g = sns.distplot(tr['limit'], kde=True, bins=200, color="blue")
# plt.show()
# plt.savefig("../images/credit_limit.png")

<img src="../images/credit_limit.png">

In [54]:
# sns.set_theme(style="darkgrid", font='serif', context='talk')
# class_0 = tr.loc[tr['default'] == 0]["limit"]
# class_1 = tr.loc[tr['default'] == 1]["limit"]
# plt.figure(figsize = (14,6))
# plt.title('Credit Limit by Default', size=16)
# sns.set_theme(style="darkgrid", font='serif', context='poster')
# sns.distplot(class_1, kde=True, bins=200, color="blue", label="No")
# sns.distplot(class_0, kde=True, bins=200, color="red", label="Yes")
# plt.legend(title = 'default', loc='upper right', facecolor='white')
# plt.show()
# plt.savefig("../images/credit_limit_by_default.png")

<img src="../images/credit_limit_by_default.png">

In [55]:
# sns.boxplot(x="gender", hue="default", y="limit", data=tr, palette="tab10")
# plt.legend(bbox_to_anchor=(1.01, 1), borderaxespad=0)
# plt.tight_layout()
# plt.title("Gender vs. Credit Limit for Defaulters and Non-defaulters", size=14)
# plt.savefig("../images/boxplot4.png")

<img src="../images/boxplot4.png">

In [56]:
# sns.boxplot(x='education', hue='default', y='limit', data=tr, palette='tab10')
# plt.legend(bbox_to_anchor=(1.01, 1), borderaxespad=0)
# plt.tight_layout()
# plt.title("Education vs. Credit Limit for Defaulters and Non-defaulters", size=14)
# plt.savefig("../images/boxplot5.png")

<img src="../images/boxplot5.png">

In [57]:
# sns.boxplot(x='marriage', hue='default', y='limit', data=tr, palette='tab10')
# plt.legend(bbox_to_anchor=(1.01, 1), borderaxespad=0)
# plt.tight_layout()
# plt.title("Marriage Status vs. Credit Limit for Defaulters and Non-defaulters", size=14)
# plt.savefig("../images/boxplot6.png")

<img src="../images/boxplot6.png">

In [58]:
sex = tr.groupby(['gender', 'default']).size().unstack(1)
sex

default,0,1
gender,,
1,6272,2019
2,10072,2637


In [59]:
# sns.set_theme(style="darkgrid", font='serif', context='poster')
# sex.plot(kind="bar", stacked=True)
# plt.legend(bbox_to_anchor=(1.01, 1), borderaxespad=0)
# plt.title("Distribution Count for Gender and Default Status",size=14)
# plt.savefig("../images/stacked_bar.png")

<img src="../images/stacked_bar.png">

In [60]:
# sns.set_theme(style="darkgrid", font='serif', context='talk')
# sns.boxplot(x='default', y='age', data=tr, palette='tab10')
# plt.title("Distribution of Default Status by Age", size=14)
# plt.savefig("../images/boxplot1.png")

<img src="../images/boxplot1.png">

In [61]:
education = tr.groupby(['education', 'default']).size().unstack(1)
education

default,0,1
education,,
1,6013,1424
2,7408,2341
3,2626,866
4,297,25


In [62]:
# education.plot(kind="bar", stacked=True)
# plt.title("Distribution Count of Educational Level and Default Status", size=14)
# plt.savefig("../data/stacked_bar2.png")

<img src="../images/stacked_bar2.png">

In [63]:
# sns.boxplot(x='default', y='age', hue='education', data=tr, palette='tab10')
# plt.legend(bbox_to_anchor=(1.01, 1), borderaxespad=0)
# plt.tight_layout()
# plt.title("Default Status vs. Age for Different Educational Status", size=14)
# plt.savefig("../images/boxplot3.png")

<img src="../images/boxplot3.png">

In [64]:
marriage = tr.groupby(['marriage', 'default']).size().unstack(1)
marriage

default,0,1
marriage,,
1,7354,2258
2,8778,2336
3,212,62


In [65]:
# sns.boxplot(x='default', y='age', hue='marriage', data=tr, palette='tab10')
# plt.legend(bbox_to_anchor=(1.01, 1), borderaxespad=0)
# plt.tight_layout()
# plt.savefig("../images/boxplot2.png")

<img src="../images/boxplot2.png">

In [66]:
# marriage.plot(kind="bar", stacked=True)
# plt.title("Distribution of Default Status for Marital Status", size=14)
# plt.savefig("../images/stacked_bar3.png")

<img src="../images/stacked_bar3.png">

In [67]:
# sns.set_theme(style="darkgrid", font='serif', context='notebook')
# g = sns.FacetGrid(tr, col='gender', row="marriage", hue='default', sharey=False)
# g.map(plt.hist, 'age', alpha=0.3, bins=50)
# g.add_legend()
# g.savefig("../images/marriage_gender.png")

<img src="../images/marriage_gender.png">

In [68]:
# sns.set_theme(style="darkgrid", font='serif', context='talk')
# g = sns.pairplot(tr[['gender', 'education', 'marriage', 'limit', 'age', 'default']], 
#              hue="default", 
#              diag_kind="kde",
#              size=4);
# g.savefig("../images/pairplot1.png")

<img src="../images/pairplot1.png">

In [69]:
# sns.set_theme(style="darkgrid", font='serif', context='poster')
# g = sns.pairplot(tr[['behind1', 'behind2', 'behind3', 'behind4', 'behind5', 'behind6', 'default']], 
#              hue="default", 
#              diag_kind="kde",
#              size=4)
# g.savefig("../images/pairplot2.png")

<img src="../images/pairplot2.png">


In [70]:
# plt.style.use("fivethirtyeight")
# sns.set_theme(style="darkgrid", font='serif', context='poster')
# g = sns.pairplot(tr[['billed1', 'billed2', 'billed3', 'billed4', 'billed5', 'billed6', 'default']], 
#              hue="default", 
#              diag_kind="kde",
#              size=4)
# g.savefig("../images/pairplot3.png")

<img src="../images/pairplot3.png">

In [71]:
# sns.set_theme(style="darkgrid", font='serif', context='talk')
# g = sns.pairplot(tr[['paid1', 'paid2', 'paid3', 'paid4', 'paid5', 'paid6', 'default']], 
#              hue="default", 
#              diag_kind="kde",
#              size=4)
# g.savefig("../images/pairplot4.png")

<img src="../images/pairplot4.png">

In [72]:
# plt.style.use("fivethirtyeight")
# sns.set_theme(style="darkgrid", font='serif', context='paper')
# plt.figure(figsize = (20,16))
# plt.title('Pearson Correlation of Features', y = 1.05, size = 20)
# # mask = np.zeros(tr.corr().shape, dtype=bool)
# # mask[np.triu_indices(len(mask))] = True
# g = sns.heatmap(tr.corr(), cmap='RdBu', annot=True, square=True, linecolor='white', linewidths=0.2, fmt=".1f")
# plt.savefig("../images/correlation_matrix.png")

# <a id='7'>Feature Engineering</a>

In [73]:
tr.head()

,limit,gender,education,marriage,age,behind1,behind2,behind3,behind4,behind5,behind6,billed1,billed2,billed3,billed4,billed5,billed6,paid1,paid2,paid3,paid4,paid5,paid6,default
0,1790.26,2,2,1,44,0,0,0,0,0,0,1631.93,1500.45,1278.35,800.60,847.12,981.81,107.99,179.13,107.42,107.42,179.03,33.08,0
1,5728.83,2,3,1,46,-1,-1,-1,0,-1,-1,891.69,83.71,173.87,147.77,143.04,30.15,83.89,173.87,35.81,143.04,30.15,942.14,0
2,3580.52,2,2,1,47,-1,-1,-1,-1,-1,-2,238.68,238.68,0.00,224.50,-14.18,-14.18,238.68,0.00,224.50,0.00,0.00,0.00,1
3,6086.88,2,2,1,29,0,0,0,0,0,0,2831.87,2240.51,2267.08,2288.06,1557.71,1575.25,80.02,89.26,92.56,60.26,68.07,75.58,0
4,5370.78,2,1,2,33,-2,-2,-2,-2,-2,-2,873.40,961.26,1170.90,1198.01,995.38,80.96,966.99,1171.37,1198.58,995.67,80.96,6067.73,0


In [74]:
data = [tr, val]

# create features for demographic variables
for d in data:
    d['age_bin'] = 0
    d.loc[((d['age'] > 20) & (d['age'] < 30)) , 'age_bin'] = 1
    d.loc[((d['age'] >= 30) & (d['age'] < 60)) , 'age_bin'] = 2
    d.loc[((d['age'] >= 60) & (d['age'] < 81)) , 'age_bin'] = 3
    # create categories for single, married, divorced males and females
    d['gen-mar'] = d['gender'] + d['marriage']
    # create categories for young, middle age and senior males and females
    d['gen-age'] = d['gender'] + d['age_bin']

# feature for credit use percentage: fraction of estimated available balance based on what is billed per month
# (credit limit - monthly billed amount) / credit limit
for d in data:
    d['avail6'] = (d.limit - d.billed6) / d.limit
    d['avail5'] = (d.limit - d.billed5) / d.limit
    d['avail4'] = (d.limit - d.billed4) / d.limit
    d['avail3'] = (d.limit - d.billed3) / d.limit
    d['avail2'] = (d.limit - d.billed2) / d.limit
    d['avail1'] = (d.limit - d.billed1) / d.limit
    d['avg_av'] = (d.avail1 + d.avail2 + d.avail3 + d.avail4 + d.avail5 + d.avail6) / 6

# create a feature that indicates whether a client has had a delayed payment or not
def delayed_payment(d):
    if (d.behind1 > 0) or (d.behind2 > 0) or (d.behind3 > 0) or (d.behind4 > 0) or (d.behind5 > 0) or (d.behind6 > 0):
        return 1
    else:
        return 0
for d in data:
    d['delayed'] = d.apply(delayed_payment, axis=1)

# create feature for the total number of months with delayed payment status for a particular client
def total_months_with_delayed_payments(d):
    count = 0
    if (d.behind1 > 0):
        count += 1
    if (d.behind2 > 0):
        count += 1
    if (d.behind3 > 0):
        count += 1
    if (d.behind4 > 0):
        count += 1
    if (d.behind5 > 0):
        count += 1
    if (d.behind6 > 0):
        count += 1
    return count
for d in data:
    d['latemths'] = d.apply(total_months_with_delayed_payments, axis=1)

# the ratio of amount paid and amount billed
for d in data:
    d['pperb1'] = d.paid1 / d.billed2
    d['pperb2'] = d.paid2 / d.billed3
    d['pperb3'] = d.paid3 / d.billed4
    d['pperb4'] = d.paid4 / d.billed5
    d['pperb5'] = d.paid5 / d.billed6

# remove any infinity and NaN values
datasets = ['pperb1', 'pperb2', 'pperb3', 'pperb4', 'pperb5']
for data in datasets:
    tr.replace({data: {np.inf: 0, np.nan: 0}}, inplace=True)
    val.replace({data: {np.inf: 0, np.nan: 0}}, inplace=True)


In [77]:
tr.head()

,limit,gender,education,marriage,age,behind1,behind2,behind3,behind4,behind5,behind6,billed1,billed2,billed3,billed4,billed5,billed6,paid1,paid2,paid3,paid4,paid5,paid6,default,age_bin,gen-mar,gen-age,avail6,avail5,avail4,avail3,avail2,avail1,avg_av,delayed,latemths,pperb1,pperb2,pperb3,pperb4,pperb5
0,1790.26,2,2,1,44,0,0,0,0,0,0,1631.93,1500.45,1278.35,800.60,847.12,981.81,107.99,179.13,107.42,107.42,179.03,33.08,0,2,3,4,0.451582,0.526817,0.552802,0.285942,0.161882,0.088440,0.344578,0,0,0.071972,0.140126,0.134174,0.126806,0.182347
1,5728.83,2,3,1,46,-1,-1,-1,0,-1,-1,891.69,83.71,173.87,147.77,143.04,30.15,83.89,173.87,35.81,143.04,30.15,942.14,0,2,3,4,0.994737,0.975032,0.974206,0.969650,0.985388,0.844350,0.957227,0,0,1.002150,1.000000,0.242336,1.000000,1.000000
2,3580.52,2,2,1,47,-1,-1,-1,-1,-1,-2,238.68,238.68,0.00,224.50,-14.18,-14.18,238.68,0.00,224.50,0.00,0.00,0.00,1,2,3,4,1.003960,1.003960,0.937300,1.000000,0.933339,0.933339,0.968650,0,0,1.000000,0.000000,1.000000,-0.000000,-0.000000
3,6086.88,2,2,1,29,0,0,0,0,0,0,2831.87,2240.51,2267.08,2288.06,1557.71,1575.25,80.02,89.26,92.56,60.26,68.07,75.58,0,1,3,3,0.741206,0.744087,0.624100,0.627546,0.631912,0.534758,0.650602,0,0,0.035715,0.039372,0.040453,0.038685,0.043212
4,5370.78,2,1,2,33,-2,-2,-2,-2,-2,-2,873.40,961.26,1170.90,1198.01,995.38,80.96,966.99,1171.37,1198.58,995.67,80.96,6067.73,0,2,4,4,0.984926,0.814668,0.776939,0.781987,0.821020,0.837379,0.836153,0,0,1.005961,1.000401,1.000476,1.000291,1.000000


In [78]:
# plt.style.use("fivethirtyeight")
# sns.set_theme(style="darkgrid", font='serif', context='paper')
# plt.figure(figsize = (20,16))
# plt.title('Pearson Correlation of Features', y = 1.05, size = 20)
# g = sns.heatmap(tr.corr(), cmap='RdBu', square=True, linecolor='white', linewidths=0.2)
# plt.savefig("../images/correlation_matrix_2.png")


<img src="../images/correlation_matrix_2.png">

In [79]:
tr.head()

,limit,gender,education,marriage,age,behind1,behind2,behind3,behind4,behind5,behind6,billed1,billed2,billed3,billed4,billed5,billed6,paid1,paid2,paid3,paid4,paid5,paid6,default,age_bin,gen-mar,gen-age,avail6,avail5,avail4,avail3,avail2,avail1,avg_av,delayed,latemths,pperb1,pperb2,pperb3,pperb4,pperb5
0,1790.26,2,2,1,44,0,0,0,0,0,0,1631.93,1500.45,1278.35,800.60,847.12,981.81,107.99,179.13,107.42,107.42,179.03,33.08,0,2,3,4,0.451582,0.526817,0.552802,0.285942,0.161882,0.088440,0.344578,0,0,0.071972,0.140126,0.134174,0.126806,0.182347
1,5728.83,2,3,1,46,-1,-1,-1,0,-1,-1,891.69,83.71,173.87,147.77,143.04,30.15,83.89,173.87,35.81,143.04,30.15,942.14,0,2,3,4,0.994737,0.975032,0.974206,0.969650,0.985388,0.844350,0.957227,0,0,1.002150,1.000000,0.242336,1.000000,1.000000
2,3580.52,2,2,1,47,-1,-1,-1,-1,-1,-2,238.68,238.68,0.00,224.50,-14.18,-14.18,238.68,0.00,224.50,0.00,0.00,0.00,1,2,3,4,1.003960,1.003960,0.937300,1.000000,0.933339,0.933339,0.968650,0,0,1.000000,0.000000,1.000000,-0.000000,-0.000000
3,6086.88,2,2,1,29,0,0,0,0,0,0,2831.87,2240.51,2267.08,2288.06,1557.71,1575.25,80.02,89.26,92.56,60.26,68.07,75.58,0,1,3,3,0.741206,0.744087,0.624100,0.627546,0.631912,0.534758,0.650602,0,0,0.035715,0.039372,0.040453,0.038685,0.043212
4,5370.78,2,1,2,33,-2,-2,-2,-2,-2,-2,873.40,961.26,1170.90,1198.01,995.38,80.96,966.99,1171.37,1198.58,995.67,80.96,6067.73,0,2,4,4,0.984926,0.814668,0.776939,0.781987,0.821020,0.837379,0.836153,0,0,1.005961,1.000401,1.000476,1.000291,1.000000


In [80]:
pickle_out = open("../data/training_features.pickle","wb")
pickle.dump(tr, pickle_out)
pickle_out.close()

In [81]:
pickle_out = open("../data/validate_features.pickle","wb")
pickle.dump(val, pickle_out)
pickle_out.close()

# <a id='7'>Post-Feature Selection</a>

In [82]:
tr.head()

,limit,gender,education,marriage,age,behind1,behind2,behind3,behind4,behind5,behind6,billed1,billed2,billed3,billed4,billed5,billed6,paid1,paid2,paid3,paid4,paid5,paid6,default,age_bin,gen-mar,gen-age,avail6,avail5,avail4,avail3,avail2,avail1,avg_av,delayed,latemths,pperb1,pperb2,pperb3,pperb4,pperb5
0,1790.26,2,2,1,44,0,0,0,0,0,0,1631.93,1500.45,1278.35,800.60,847.12,981.81,107.99,179.13,107.42,107.42,179.03,33.08,0,2,3,4,0.451582,0.526817,0.552802,0.285942,0.161882,0.088440,0.344578,0,0,0.071972,0.140126,0.134174,0.126806,0.182347
1,5728.83,2,3,1,46,-1,-1,-1,0,-1,-1,891.69,83.71,173.87,147.77,143.04,30.15,83.89,173.87,35.81,143.04,30.15,942.14,0,2,3,4,0.994737,0.975032,0.974206,0.969650,0.985388,0.844350,0.957227,0,0,1.002150,1.000000,0.242336,1.000000,1.000000
2,3580.52,2,2,1,47,-1,-1,-1,-1,-1,-2,238.68,238.68,0.00,224.50,-14.18,-14.18,238.68,0.00,224.50,0.00,0.00,0.00,1,2,3,4,1.003960,1.003960,0.937300,1.000000,0.933339,0.933339,0.968650,0,0,1.000000,0.000000,1.000000,-0.000000,-0.000000
3,6086.88,2,2,1,29,0,0,0,0,0,0,2831.87,2240.51,2267.08,2288.06,1557.71,1575.25,80.02,89.26,92.56,60.26,68.07,75.58,0,1,3,3,0.741206,0.744087,0.624100,0.627546,0.631912,0.534758,0.650602,0,0,0.035715,0.039372,0.040453,0.038685,0.043212
4,5370.78,2,1,2,33,-2,-2,-2,-2,-2,-2,873.40,961.26,1170.90,1198.01,995.38,80.96,966.99,1171.37,1198.58,995.67,80.96,6067.73,0,2,4,4,0.984926,0.814668,0.776939,0.781987,0.821020,0.837379,0.836153,0,0,1.005961,1.000401,1.000476,1.000291,1.000000


In [83]:
train3 = tr[['limit', 'behind1', 'paid2', 'delayed', 'latemths', 'age', 'behind2', 'billed1', 'avg_av', 'avail1', 'default']]
validate3 = val[['limit', 'behind1', 'paid2', 'delayed', 'latemths', 'age', 'behind2', 'billed1', 'avg_av', 'avail1', 'default']]

In [84]:
train3.head()

,limit,behind1,paid2,delayed,latemths,age,behind2,billed1,avg_av,avail1,default
0,1790.26,0,179.13,0,0,44,0,1631.93,0.344578,0.088440,0
1,5728.83,-1,173.87,0,0,46,-1,891.69,0.957227,0.844350,0
2,3580.52,-1,0.00,0,0,47,-1,238.68,0.968650,0.933339,1
3,6086.88,0,89.26,0,0,29,0,2831.87,0.650602,0.534758,0
4,5370.78,-2,1171.37,0,0,33,-2,873.40,0.836153,0.837379,0


In [85]:
validate3.head()

,limit,behind1,paid2,delayed,latemths,age,behind2,billed1,avg_av,avail1,default
0,1074.16,0,71.61,0,0,25,0,317.38,0.602052,0.704532,0
1,5370.78,0,151.64,0,0,26,0,4895.86,0.293715,0.088427,0
2,2506.36,0,111.43,0,0,32,0,2510.73,0.005217,-0.001744,0
3,4654.68,0,64.74,0,0,49,0,740.38,0.883482,0.840939,0
4,1790.26,0,53.71,1,1,36,0,3373.85,0.188227,-0.884559,1


In [86]:
pickle_out = open("../data/training_model.pickle","wb")
pickle.dump(train3, pickle_out)
pickle_out.close()

In [87]:
pickle_out = open("../data/validate_model.pickle","wb")
pickle.dump(validate3, pickle_out)
pickle_out.close()